In [1]:
import pandas as pd
import sqlite3
import warnings

In [2]:
# Ignorar o aviso de UserWarning do pandas sobre o driver do SQLite
warnings.filterwarnings('ignore', category=UserWarning)

In [39]:
# --- Conexão ---
db_path = 'data/pipeline_agricultura.db'

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [4]:
try:
    conn = sqlite3.connect(db_path)
    print(f"Conexão com banco de dados '{db_path}' estabelecida.")
except sqlite3.OperationalError:
    print(f"Erro: Não foi possível encontrar o banco de dados '{db_path}'.")
    exit()

Conexão com banco de dados 'data/pipeline_agricultura.db' estabelecida.


In [5]:
# ==========================================
# QUERY 1: Total de Registros na Tabela Fato
# ==========================================
query1 = """
SELECT COUNT(*) as total_registros
FROM fato_producao
"""
resultado1 = pd.read_sql_query(query1, conn)
print("\n--- QUERY 1: Total de Registros na Tabela Fato ---")
print(f"Total de registros: {resultado1['total_registros'].values[0]}")


--- QUERY 1: Total de Registros na Tabela Fato ---
Total de registros: 55682


In [ ]:
# ==========================================
# QUERY 2: Top 10 Regiões por Produção Total
# ==========================================
query2 = """
SELECT
    dr."State District" as Regiao,
    SUM(fp.Production) as Producao_Total_Acumulada,
    AVG(fp.Yield) as Rendimento_Medio
FROM fato_producao fp
JOIN dim_regiao dr ON fp.id_regiao = dr.id_regiao
GROUP BY dr."State District"
ORDER BY Producao_Total_Acumulada DESC
LIMIT 10
"""
top_regioes = pd.read_sql_query(query2, conn)
print("\n--- QUERY 2: Top 10 Regiões por Produção Total (Usando Dimensão) ---")
print(top_regioes.to_string())


--- QUERY 2: Top 10 Regiões por Produção Total (Usando Dimensão) ---
                           Regiao  Producao_Total_Acumulada  Rendimento_Medio
0                PUNJAB - SANGRUR                64471200.0          4.102346
1               PUNJAB - LUDHIANA                51930300.0          4.112727
2              PUNJAB - FIROZEPUR                51509000.0          4.038448
3                PUNJAB - PATIALA                46631200.0          3.932414
4   WEST BENGAL - PURBA BARDHAMAN                41215783.0          2.552901
5               PUNJAB - AMRITSAR                40191300.0          3.583125
6  ANDHRA PRADESH - WEST GODAVARI                38298656.0          4.488315
7    WEST BENGAL - MEDINIPUR WEST                37247096.0          2.432576
8               PUNJAB - BATHINDA                34709400.0          3.898429
9                   PUNJAB - MOGA                33261700.0          4.148548


In [10]:
# ==========================================
# QUERY 3: Análise Temporal (Produção por Ano)
# ==========================================
query3 = """
SELECT
    YEAR as Ano,
    SUM(Production) as Producao_Total_Anual,
    AVG(Yield) as Rendimento_Medio_Anual
FROM fato_producao
GROUP BY YEAR
ORDER BY YEAR ASC
"""
producao_anual = pd.read_sql_query(query3, conn)
print("\n--- QUERY 3: Produção Total e Rendimento Médio por Ano ---")
print(producao_anual.to_string())


--- QUERY 3: Produção Total e Rendimento Médio por Ano ---
     Ano  Producao_Total_Anual  Rendimento_Medio_Anual
0   1997           137692535.0                5.360283
1   1998           165596180.0                1.775296
2   1999           172488939.0                1.873804
3   2000           166229480.0                1.818633
4   2001           177029932.0                1.939079
5   2002           149847723.0                1.757232
6   2003           176835284.0                1.907646
7   2004           168845917.0                1.925305
8   2005           172944883.0                1.881992
9   2006           186103686.0                1.970302
10  2007           195674011.0                2.027180
11  2008           199454647.0                2.060063
12  2009           187646346.0                1.957913
13  2010           214121717.0                2.208977
14  2011           232001663.0                2.329956
15  2012           232932707.0                2.336708
16  2

In [19]:
# ==========================================
# QUERY 4: Resumo de Desempenho por Cultura
# ==========================================

query4 = """
SELECT
    dc.Crop as Cultura,
    COUNT(DISTINCT fp.id_regiao) as Total_Regioes_Produtoras,
    SUM(fp.Production) as Producao_Total,
    AVG(fp.Yield) as Rendimento_Medio_Geral
FROM fato_producao fp
JOIN dim_cultura dc ON fp.id_cultura = dc.id_cultura
GROUP BY dc.Crop
ORDER BY Producao_Total DESC
"""
segmento_cultura = pd.read_sql_query(query4, conn)
print("\n--- QUERY 4: Resumo de Desempenho por Cultura (Usando Dimensões) ---")
print(segmento_cultura.to_string())


--- QUERY 4: Resumo de Desempenho por Cultura (Usando Dimensões) ---
  Cultura  Total_Regioes_Produtoras  Producao_Total  Rendimento_Medio_Geral
0    RICE                       729      2101070021                       2
1   WHEAT                       648      1874005141                       2
2   MAIZE                       722       411334265                       3
3  BARLEY                       352        33244358                       2


In [25]:
# ==========================================
# QUERY 5: Top 5 Regiões Mais Estáveis (MAIZE)
# ==========================================
print("\n--- QUERY 5: Top 5 Regiões Mais Estáveis para MAIZE (Volatilidade) ---")
query5 = """
SELECT
    "State District" as Regiao,
    Crop,
    "Coef_Variacao_Rendimento (%)" as CV_Rendimento
FROM volatilidade_rendimento_regiao
WHERE
    Crop = 'MAIZE' AND
    "Coef_Variacao_Rendimento (%)" > 0
ORDER BY CV_Rendimento ASC
LIMIT 5
"""
try:
    df_resultado5 = pd.read_sql_query(query5, conn)
    print(df_resultado5.to_string())
except Exception as e:
    print(f"ERRO na QUERY 5: {e}")


--- QUERY 5: Top 5 Regiões Mais Estáveis para MAIZE (Volatilidade) ---
                              Regiao   Crop  CV_Rendimento
0         ARUNACHAL PRADESH - NAMSAI  MAIZE              1
1  MEGHALAYA - SOUTH WEST GARO HILLS  MAIZE              1
2       MEGHALAYA - NORTH GARO HILLS  MAIZE              1
3          ARUNACHAL PRADESH - SIANG  MAIZE              1
4     MEGHALAYA - WEST JAINTIA HILLS  MAIZE              1


In [ ]:
# ==========================================
# QUERY 6: Regiões com Melhor Desempenho vs. Média Nacional (RICE)
# ==========================================
print("\n--- QUERY 6: Top 5 Regiões com Rendimento Acima da Média Nacional (RICE) ---")
query6 = """
SELECT
    "State District",
    Rendimento_Medio_Local,
    Rendimento_Medio_Nacional,
    "Diferenca_vs_Media (%)"
FROM benchmark_regional_rendimento
WHERE
    Crop = 'RICE'
ORDER BY "Diferenca_vs_Media (%)" DESC
LIMIT 5
"""
try:
    df_resultado6 = pd.read_sql_query(query6, conn)
    print(df_resultado6.to_string())
except Exception as e:
    print(f"ERRO na QUERY 6: {e}")


--- QUERY 6: Top 5 Regiões com Rendimento Acima da Média Nacional (RICE) ---
            State District  Rendimento_Medio_Local  Rendimento_Medio_Nacional  Diferenca_vs_Media (%)
0   MAHARASHTRA - KOLHAPUR                       8                          2                     282
1  CHANDIGARH - CHANDIGARH                       5                          2                     137
2         PUNJAB - BARNALA                       5                          2                     126
3       TAMIL NADU - THENI                       5                          2                     125
4    TAMIL NADU - TIRUPPUR                       5                          2                     119


In [ ]:
# ==========================================
# QUERY 7: Tendência de Crescimento Anual (WHEAT)
# ==========================================
print("\n--- QUERY 7: Crescimento Anual do Rendimento para WHEAT (Últimos 5 Anos) ---")
query7 = """
SELECT
    YEAR,
    Rendimento_Medio,
    "Crescimento_Anual_Rendimento (%)"
FROM tendencia_anual_rendimento
WHERE
    Crop = 'WHEAT'
ORDER BY YEAR DESC
LIMIT 5
"""
try:
    df_resultado7 = pd.read_sql_query(query7, conn)
    print(df_resultado7.to_string())
except Exception as e:
    print(f"ERRO na QUERY 7: {e}")


--- QUERY 7: Crescimento Anual do Rendimento para WHEAT (Últimos 5 Anos) ---
   YEAR  Rendimento_Medio  Crescimento_Anual_Rendimento (%)
0  2018                 3                                 1
1  2017                 3                                 2
2  2016                 3                                14
3  2015                 2                                 7
4  2014                 2                               -11


In [34]:
# ==========================================
# QUERY 8: Perfil Climático da Região Mais Produtiva (RICE)
# ==========================================
print("\n--- QUERY 8: Perfil Climático da Região Mais Produtiva (RICE) ---")
query8 = """
SELECT
    t1."State District",
    t1.Rendimento_Medio,
    t1.Precipitacao_Jun_Sep_Media,
    t1.Temp_Media_JunSep
FROM perfil_climatico_regiao_cultura t1
WHERE
    t1.Crop = 'RICE'
ORDER BY t1.Rendimento_Medio DESC
LIMIT 1
"""
try:
    df_resultado8 = pd.read_sql_query(query8, conn)
    print(df_resultado8.to_string())
except Exception as e:
    print(f"ERRO na QUERY 8: {e}")


--- QUERY 8: Perfil Climático da Região Mais Produtiva (RICE) ---
           State District  Rendimento_Medio  Precipitacao_Jun_Sep_Media  Temp_Media_JunSep
0  MAHARASHTRA - KOLHAPUR                 8                         857                 28


In [ ]:
# ==========================================
# QUERY 9: Desempenho Sazonal (RICE)
# ==========================================
print("\n--- QUERY 9: Rendimento Médio de RICE por Temporada (Season) ---")
query9 = """
SELECT
    Season,
    Rendimento_Medio,
    Precipitacao_Media_Jun_Sep,
    Temp_Media_Jun_Sep
FROM analise_sazonal_clima
WHERE
    Crop = 'RICE'
ORDER BY Rendimento_Medio DESC
"""
try:
    df_resultado9 = pd.read_sql_query(query9, conn)
    print(df_resultado9.to_string())
except Exception as e:
    print(f"ERRO na QUERY 9: {e}")


--- QUERY 9: Rendimento Médio de RICE por Temporada (Season) ---
       Season  Rendimento_Medio  Precipitacao_Media_Jun_Sep  Temp_Media_Jun_Sep
0        RABI                 3                         862                  28
1      SUMMER                 2                         859                  28
2      KHARIF                 2                         857                  28
3      WINTER                 2                         861                  28
4      AUTUMN                 1                         859                  28
5  WHOLE YEAR                 1                         758                  28


In [36]:
# ==========================================
# QUERY 10: Regiões com Maior Volatilidade (WHEAT)
# ==========================================
print("\n--- QUERY 10: Top 5 Regiões com Maior Volatilidade para WHEAT ---")
query10 = """
SELECT
    "State District" as Regiao,
    Crop,
    "Coef_Variacao_Rendimento (%)" as CV_Rendimento,
    Contagem_Registros
FROM volatilidade_rendimento_regiao
WHERE
    Crop = 'WHEAT'
ORDER BY CV_Rendimento DESC
LIMIT 5
"""
try:
    df_resultado10 = pd.read_sql_query(query10, conn)
    print(df_resultado10.to_string())
except Exception as e:
    print(f"ERRO na QUERY 10: {e}")


--- QUERY 10: Top 5 Regiões com Maior Volatilidade para WHEAT ---
                      Regiao   Crop  CV_Rendimento  Contagem_Registros
0       TELANGANA - ADILABAD  WHEAT             91                   6
1          ODISHA - GAJAPATI  WHEAT             86                   5
2  ANDHRA PRADESH - CHITTOOR  WHEAT             84                   4
3   ANDHRA PRADESH - KURNOOL  WHEAT             67                  20
4     KARNATAKA - SHIVAMOGGA  WHEAT             66                   5


In [40]:
# ==========================================
# QUERY 11: Análise de Produção Anual (MAIZE)
# ==========================================
print("\n--- QUERY 11: Produção Total e Rendimento Ponderado de MAIZE por Ano ---")
query11 = """
SELECT
    YEAR,
    Producao_Total,
    Rendimento_Ponderado
FROM producao_anual_cultura
WHERE
    Crop = 'MAIZE'
ORDER BY YEAR DESC
"""
try:
    df_resultado11 = pd.read_sql_query(query11, conn)
    print(df_resultado11.to_string())
except Exception as e:
    print(f"ERRO na QUERY 11: {e}")


--- QUERY 11: Produção Total e Rendimento Ponderado de MAIZE por Ano ---
    YEAR  Producao_Total  Rendimento_Ponderado
0   2018    28447793.000                 3.129
1   2017    33122757.000                 3.533
2   2016    27612177.000                 2.935
3   2015    22721124.000                 2.628
4   2014    24914516.000                 2.756
5   2013    26279747.000                 3.037
6   2012    23344091.000                 2.733
7   2011    22225075.000                 2.574
8   2010    22647927.000                 2.696
9   2009    16420185.000                 2.031
10  2008    18663761.000                 2.465
11  2007    19358633.000                 2.441
12  2006    14730786.000                 1.919
13  2005    14764110.000                 2.037
14  2004    13823327.000                 1.905
15  2003    14639032.000                 2.054
16  2002    11025210.000                 1.678
17  2001    12673331.000                 1.997
18  2000    11977402.000         

In [41]:
# ==========================================
# QUERY 12: Regiões com Pior Desempenho vs. Média Nacional (BARLEY)
# ==========================================
print("\n--- QUERY 12: Top 5 Regiões com Rendimento Abaixo da Média Nacional (BARLEY) ---")
query12 = """
SELECT
    "State District",
    Rendimento_Medio_Local,
    Rendimento_Medio_Nacional,
    "Diferenca_vs_Media (%)"
FROM benchmark_regional_rendimento
WHERE
    Crop = 'BARLEY'
ORDER BY "Diferenca_vs_Media (%)" ASC
LIMIT 5
"""
try:
    df_resultado12 = pd.read_sql_query(query12, conn)
    print(df_resultado12.to_string())
except Exception as e:
    print(f"ERRO na QUERY 12: {e}")


--- QUERY 12: Top 5 Regiões com Rendimento Abaixo da Média Nacional (BARLEY) ---
                 State District  Rendimento_Medio_Local  Rendimento_Medio_Nacional  Diferenca_vs_Media (%)
0   MADHYA PRADESH - CHHINDWARA                   0.190                      1.932                 -90.164
1  JAMMU AND KASHMIR - SRINAGAR                   0.250                      1.932                 -87.058
2  JAMMU AND KASHMIR - KISHTWAR                   0.314                      1.932                 -83.755
3    JAMMU AND KASHMIR - RAMBAN                   0.320                      1.932                 -83.434
4      JAMMU AND KASHMIR - DODA                   0.324                      1.932                 -83.219


In [ ]:
conn.close()